# TenSEAL CKKS - Introduction

Homomorphic Encryption  is an encryption technique that allows computations to be made on ciphertexts and generates results that when decrypted, correspond to the results of the same computations made on plaintexts. 

**[CKKS - Cheon-Kim-Kim-Song](https://www.inferati.com/blog/fhe-schemes-ckks)** which is Homomorphic Encryption for Arithmetic of Approximate Numbers (HEAAN), was proposed to offer homomorphic computation on real numbers. The main idea is to consider the noise, a.k.a. error $ e$ , which is introduced in Ring-Learning with Errors (Ring-LWE) based FHE schemes for security purposes, as part of the message $\mu $ (which we call here payload) we want to encrypt. The payload and the noise are combined to generate the plaintext ($\mu + e$) that we encrypt.

Next we will look at the most important object of the library, the TenSEALContext for CKKS HE Scheme :

In [1]:
import tenseal as ts
import numpy as np

In [2]:
# Setup TenSEAL context
context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=8192,
            coeff_mod_bit_sizes=[60, 40, 40, 60]
          )
"""
Genrate Galois Keys : Evaluation keys for the homomorphic rotation operation,
which is the cyclic shift operations for rows of the encrypted matrix in one
ciphertext of the BFV scheme and for encrypted message vector in that of the 
CKKS scheme.
"""
context.generate_galois_keys()
context.global_scale = 2**40

In [3]:
# Add two vectors
v1 = [5, 1, 2, 3, 4]
v2 = [4, 3, 2, 1, 5]

In [4]:
# Encrypt the vectors using the CKKS scheme
enc_v1 = ts.ckks_vector(context, v1)
enc_v2 = ts.ckks_vector(context, v2)

## Addition

In [5]:
add_result = enc_v1 + enc_v2
add_result.decrypt() # ~ [9,4,4,4,9]

[9.000000003111897,
 3.999999997718233,
 4.000000000318537,
 4.000000002821806,
 8.999999999744613]

### Time taken to perform addition

In [6]:
print("Homomorphic Addition takes:")
he_add =  %timeit -o (enc_v1 + enc_v2)
print("Vector Addition takes:")
v_add = %timeit -o (np.array(v1) + np.array(v2))
res = he_add.best / v_add.best

print("Vector Addition is {} times faster than Homomorphic Addition".format(res))

Homomorphic Addition takes:
59.2 µs ± 626 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Vector Addition takes:
2.09 µs ± 26.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Vector Addition is 28.247359974657833 times faster than Homomorphic Addition


## Subtraction

In [7]:
subp_result = np.array(v1) - np.array(v2)
subp_result

array([ 1, -2,  0,  2, -1])

In [8]:
sub_result = enc_v1 - enc_v2
sub_result.decrypt()

[0.9999999966010994,
 -2.0000000002092166,
 -5.7121862795384004e-11,
 2.0000000032285046,
 -0.9999999998236382]

### Time taken to perform Subtraction

In [9]:
print("Homomorphic Subtraction takes:")
he_sub =  %timeit -o (enc_v2 - enc_v1)
print("Vector Subtraction takes:")
v_sub = %timeit -o (np.array(v2) - np.array(v1))

res = he_sub.best / v_sub.best
print("Vector Subtraction is {} times faster than Homomorphic Subtraction".format(res))

Homomorphic Subtraction takes:
53.4 µs ± 974 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Vector Subtraction takes:
2.1 µs ± 44.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Vector Subtraction is 25.3359051117995 times faster than Homomorphic Subtraction


## Negation

In [10]:
neg_result = -enc_v1
neg_result.decrypt()

[-4.999999999856498,
 -0.9999999987545065,
 -2.0000000001307074,
 -3.0000000030251552,
 -3.9999999999604876]

### Time taken to perform Negation

In [11]:
print("Homomorphic Negation takes:")
he_neg =  %timeit -o (-enc_v1)
print("Vector Subtraction takes:")
v_neg = %timeit -o (-np.array(v1))

res = he_neg.best / v_neg.best
print("Vector Negation is {} times faster than Homomorphic Negation".format(res))

Homomorphic Negation takes:
52.5 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Vector Subtraction takes:
1.16 µs ± 12.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
Vector Negation is 45.239837229104964 times faster than Homomorphic Negation


## Power

In [12]:
power_res = enc_v1 ** 4
power_res._decrypt()

[625.0005866668016,
 1.0000009368078793,
 16.00001502308851,
 81.00007640350648,
 256.00024030782794]

### Time taken to calculate exponents and powers

In [13]:
print("Homomorphic Negation takes:")
he_pow =  %timeit -o (enc_v1 ** 4)
print("Vector Subtraction takes:")
v_pow = %timeit -o (np.array(v1) ** 4)

res = he_pow.best / v_pow.best
print("Vector Negation is {} times faster than Homomorphic Negation".format(res))

Homomorphic Negation takes:
6.29 ms ± 35.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Vector Subtraction takes:
1.76 µs ± 12.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
Vector Negation is 3591.9838686123685 times faster than Homomorphic Negation


## Dot Product

In [14]:
result = enc_v1.dot(enc_v2)
result.decrypt() # ~ [50]

[50.00000598789758]

### Time taken to perform Dot Product

In [15]:
print("Homomorphic Dot Product takes:")
he_dp =  %timeit -o (enc_v1.dot(enc_v2))
print("Vector Dot Product takes:")
v_dp = %timeit -o (np.array(v1).dot(np.array(v2)))
res = he_dp.best / v_dp.best

print("Vector Dot Product is {} times faster than Homomorphic Dot Product".format(res))

Homomorphic Dot Product takes:
9.61 ms ± 84.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Vector Dot Product takes:
2.39 µs ± 65.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Vector Dot Product is 4039.082239497987 times faster than Homomorphic Dot Product


## Matrix Multiplication

In [16]:
matrix = [
  [79, 0.15, 18],
  [21, -5, 64],
  [-18, -98, -3],
  [1, 9, 87],
  [88, 65 , 1],
]

In [17]:
result = enc_v1.matmul(matrix)
result.decrypt()

[735.0000985280301, 86.75001165200263, 413.00005566849956]

### Time taken to perform Matrix Multiplication

In [18]:
print("Homomorphic Matrix Multiplication takes:")
he_mm =  %timeit -o (enc_v1.matmul(matrix))
print("Vector Matrix Multiplication takes:")
v_mm = %timeit -o (np.matmul(v1,matrix))
res = he_mm.best / v_mm.best

print("Vector Matrix Multiplication is {} times faster than HomomorphicMatrix Multiplication".format(res))

Homomorphic Matrix Multiplication takes:
95.6 ms ± 4.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Vector Matrix Multiplication takes:
7.13 µs ± 61.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Vector Matrix Multiplication is 12776.268429061454 times faster than HomomorphicMatrix Multiplication


## Conclusion

In this notebook, we saw various capabilities and operations that are supported by CKKS scheme. These include - addition, subtraction, negation, power, multiplication, dot product, polynomial evaluation and matrix multiplication. In the future work, we can start looking into another capability of the CKKSvector that allows image block to columns operation and can be used for CNN for classification. 